In [1]:
%load_ext autoreload
%autoreload 2

# Imports

In [6]:
import numpy as np
import pandas as pd

In [7]:
from dataloading import preprocess, standard_scale

In [5]:
# columns
attack_columns = ['Attack ID', 'Card', 'Victim IP', 'Port number', 'Attack code', 'Detect count', 'Significant flag',
                   'Packet speed', 'Data speed', 'Avg packet len', 'Avg source IP count', 'Start time', 'End time', 'Whitelist flag', 'Type']
vector_columns = ['Attack ID', 'Detect count', 'Card', 'Victim IP', 'Port number', 'Attack code', 'Significant flag', 
                  'Packet speed', 'Data speed', 'Avg packet len', 'Source IP count', 'Time']

numerical_cols = ['Port number', 'Packet speed', 'Data speed', 'Avg packet len', 'Source IP count', 'victim IP num', 'time_of_day']

## Train dataset

In [9]:
train_df = pd.read_csv('../data/preprocessed/train_vectors.csv')

# scale the numerical columns
scaled_df, train_scaler = standard_scale(train_df, numerical_cols)
scaled_df.head()

,Port number,Packet speed,Data speed,Avg packet len,Source IP count,victim IP num,time_of_day,Significant flag,CLDAP,Generic UDP,...,Sentinel,IPv4 fragmentation,Suspicious traffic,SSDP,TCP Anomaly,SNMP,DNS,is_weekday,Type,is_synthetic
0,-0.817317,-0.258647,-0.114203,0.752471,-0.179160,-0.604654,0.597073,0,0,0,...,0,0,0,0,0,0,0,1,Normal traffic,False
1,-0.817317,-0.195674,0.024471,0.978049,-0.191742,-0.604474,0.670414,0,0,0,...,0,0,0,0,0,0,0,1,Normal traffic,False
2,-0.941431,-0.225965,-0.065259,0.781814,-0.191742,-0.604294,0.680810,0,0,0,...,0,0,0,0,0,0,0,1,Normal traffic,False
3,-0.941431,-0.178138,-0.008159,0.781814,-0.191742,-0.604294,0.680854,0,0,0,...,0,0,0,0,0,0,0,1,Normal traffic,False
4,-0.817317,-0.227559,-0.016316,0.941370,-0.191742,-0.604474,0.697654,0,0,0,...,0,0,0,0,0,0,0,1,Normal traffic,False


In [14]:
# save the dataset
scaled_df.to_csv('../data/preprocessed/train_vectors_scaled.csv', index = False)

## Validation dataset

In [15]:
# path to the files
attacks_path = '../data/raw/attacks.anon.validation.csv'
vectors_path = '../data/raw/vectors.anon.validation.csv'

# read the datasets
attack_df = pd.read_csv(attacks_path, names = attack_columns)
vector_df = pd.read_csv(vectors_path, names= vector_columns)

#preprocess
preprocessed_df = preprocess(vector_df, attack_df)

In [16]:
# save the dataset
preprocessed_df.to_csv('../data/preprocessed/validation_vectors.csv', index = False)

In [17]:
# scale the numerical columns
scaled_df, scaler = standard_scale(preprocessed_df, numerical_cols, scaler = train_scaler)
scaled_df.head()

,Port number,Significant flag,Packet speed,Data speed,Avg packet len,Source IP count,Type,Memcached,RPC,CHARGEN,...,SSDP,NTP,DNS,TCP Anomaly,SNMP,CoAP,RDP,victim IP num,is_weekday,time_of_day
0,-0.986564,0,-0.288938,-0.106046,0.978049,-0.191742,Normal traffic,0,0,0,...,0,0,0,0,0,0,0,10.287790,1,0.587423
1,-0.986564,0,-0.159804,0.032628,0.790984,-0.191742,Normal traffic,0,0,0,...,0,0,0,0,0,0,0,11.541849,1,0.588958
2,-0.986564,0,-0.273792,-0.089731,0.956042,-0.191742,Normal traffic,0,0,0,...,0,0,0,0,0,0,0,10.287790,1,0.589353
3,-0.986564,0,-0.051395,0.106043,0.576409,-0.191742,Normal traffic,0,0,0,...,0,0,0,0,0,0,0,10.367687,1,0.589484
4,1.448900,0,-0.237922,-0.130518,0.565405,-0.191742,Normal traffic,0,0,0,...,0,0,0,0,0,0,0,10.367687,1,0.589484


In [22]:
# save the dataset
scaled_df.to_csv('../data/preprocessed/validation_vectors_scaled.csv', index = False)

## Test dataset

In [23]:
# path to the files
attacks_path = '../data/raw/attacks.anon.test.csv'
vectors_path = '../data/raw/vectors.anon.test.csv'

In [24]:
# read the datasets
attack_df = pd.read_csv(attacks_path, names = attack_columns)
vector_df = pd.read_csv(vectors_path, names= vector_columns)

#preprocess
preprocessed_df = preprocess(vector_df, attack_df)

In [25]:
# save the dataset
preprocessed_df.to_csv('../data/preprocessed/test_vectors.csv', index = False)

In [27]:
# scale the numerical columns
scaled_df, scaler = standard_scale(preprocessed_df, numerical_cols, scaler=train_scaler)
scaled_df.head()

,Port number,Significant flag,Packet speed,Data speed,Avg packet len,Source IP count,Type,Memcached,RPC,CHARGEN,...,Generic UDP,NTP,DNS,CoAP,SSDP,WSD,High volume traffic,victim IP num,is_weekday,time_of_day
0,-0.986526,0,-0.701848,-0.708886,-1.782906,-0.194735,Normal traffic,0,0,0,...,0,0,0,0,0,0,1,-0.603550,1,-2.270616
1,-0.986600,0,-0.701849,-0.711415,-1.783652,-0.194729,Normal traffic,0,0,0,...,0,0,0,0,0,0,1,-0.603017,1,-2.270616
2,-0.986518,0,-0.701849,-0.710882,-1.782845,-0.194729,Normal traffic,0,0,0,...,0,0,0,0,0,0,1,-0.604310,1,-2.270616
3,-0.986601,0,-0.701850,-0.715274,-1.787187,-0.194418,Suspicious traffic,0,0,0,...,0,0,0,0,0,0,0,-0.604256,1,-2.270616
4,-0.986570,0,-0.701848,-0.709884,-1.782943,-0.194703,Normal traffic,0,0,0,...,0,0,0,0,0,0,1,-0.604833,1,-2.270616


In [29]:
# save the dataset
scaled_df.to_csv('../data/preprocessed/test_vectors_scaled.csv', index = False)

## Generalisation dataset

In [30]:
# path to the files
attacks_path = '../data/raw/attacks.anon.gen.csv'
vectors_path = '../data/raw/vectors.anon.gen.csv'

# read the datasets
attack_df = pd.read_csv(attacks_path, header=0)
vector_df = pd.read_csv(vectors_path, header=0)

#preprocess
preprocessed_df = preprocess(vector_df, attack_df)

In [31]:
# save the dataset
preprocessed_df.to_csv('../data/preprocessed/generalisation_vectors.csv', index = False)

In [32]:
# scale the numerical columns
scaled_df, scaler = standard_scale(preprocessed_df, numerical_cols, scaler = train_scaler)
scaled_df.head()

,Port number,Significant flag,Packet speed,Data speed,Avg packet len,Source IP count,Type,Memcached,RPC,CHARGEN,...,SSDP,NTP,DNS,TCP Anomaly,SNMP,CoAP,RDP,victim IP num,is_weekday,time_of_day
0,0.962409,1,0.599854,-0.040788,-0.302065,-0.189226,Normal traffic,0,0,0,...,0,0,0,0,0,0,0,8.411650,1,0.296251
1,0.962409,1,0.609420,-0.032630,-0.270887,-0.189226,Normal traffic,0,0,0,...,0,0,0,0,0,0,0,8.411650,1,0.296251
2,0.962409,1,0.240352,0.024471,-0.302065,-0.191742,Normal traffic,0,0,0,...,0,0,0,0,0,0,0,8.411650,1,0.297128
3,-0.986564,1,-0.009945,0.130515,0.482876,-0.191742,Normal traffic,0,0,0,...,0,0,0,0,0,0,0,14.623281,1,0.305550
4,1.256674,1,0.239555,0.432335,0.459035,-0.191742,Normal traffic,0,0,0,...,0,0,0,0,0,0,0,14.623281,1,0.305638


In [34]:
# save the dataset
scaled_df.to_csv('../data/preprocessed/generalisation_vectors_scaled.csv', index = False)